# Effectiveness of PowerPlay
## 나라별로 파워플레이(Power Play)를 언제 사용하는 것이 가장 효과적인가

→ 단순 득점률뿐 아니라 샷 품질(Points), 스톤 위치 패턴, 게임 상황(점수차)까지 포함하여 전략 효율성을 평가한다

In [282]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [257]:
competitions = pd.read_csv("Data/Competition.csv")
competitors = pd.read_csv("Data/Competitors.csv")
ends = pd.read_csv("Data/Ends.csv")
games = pd.read_csv("Data/Games.csv")
stones = pd.read_csv("Data/Stones.csv")
teams = pd.read_csv("Data/Teams.csv")

### 기본 데이터 병합

In [258]:
# Ends + Games + Teams
ends_full = (
    ends.merge(games, on=["CompetitionID","SessionID","GameID"], how="left")
            .merge(teams, on=["CompetitionID","TeamID"], how="left")
)

# Stones + Ends
stones_full = (
    stones.merge(ends_full[["CompetitionID","SessionID","GameID","EndID","TeamID","PowerPlay","NOC","Result","Winner"]],
                    on=["CompetitionID","SessionID","GameID","EndID","TeamID"], how="left")
)

In [259]:
ends_full.head(16)

,CompetitionID,SessionID,GameID,TeamID,EndID,Result,PowerPlay,GroupID,Sheet,NOC1,NOC2,ResultStr1,ResultStr2,LSFE,Winner,TeamID1,TeamID2,NOC,Name
0,0,1,1,19,1,0,NaN,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden
1,0,1,1,27,1,1,NaN,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain
2,0,1,1,19,2,2,NaN,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden
3,0,1,1,27,2,0,NaN,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain
4,0,1,1,19,3,0,NaN,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden
5,0,1,1,27,3,3,NaN,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain
6,0,1,1,19,4,1,NaN,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden
7,0,1,1,27,4,0,NaN,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain
8,0,1,1,19,5,2,NaN,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden
9,0,1,1,27,5,0,NaN,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain


In [260]:
stones_full.head(10)

,CompetitionID,SessionID,GameID,EndID,ShotID,TeamID,PlayerID,Task,Handle,Points,TimeOut,stone_1_x,stone_1_y,stone_2_x,stone_2_y,stone_3_x,stone_3_y,stone_4_x,stone_4_y,stone_5_x,stone_5_y,stone_6_x,stone_6_y,stone_7_x,stone_7_y,stone_8_x,stone_8_y,stone_9_x,stone_9_y,stone_10_x,stone_10_y,stone_11_x,stone_11_y,stone_12_x,stone_12_y,PowerPlay,NOC,Result,Winner
0,0,1,1,1,7,19,1,0,0,0,NaN,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,750.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,SWE,0,0
1,0,1,1,1,8,27,1,0,1,2,NaN,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,750.0,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,GBR,1,0
2,0,1,1,1,9,19,2,0,0,4,NaN,750.0,1916.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,0.0,0.0,750.0,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,SWE,0,0
3,0,1,1,1,16,27,2,0,0,0,NaN,963.0,1657.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,0.0,0.0,750.0,650.0,892.0,1232.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,GBR,1,0
4,0,1,1,1,17,19,2,7,0,3,NaN,963.0,1657.0,4095.0,4095.0,694.0,756.0,636.0,1080.0,0.0,0.0,0.0,0.0,750.0,650.0,4095.0,0.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,SWE,0,0
5,0,1,1,1,18,27,2,6,1,0,NaN,963.0,1657.0,4095.0,4095.0,671.0,713.0,600.0,798.0,0.0,0.0,0.0,0.0,4095.0,0.0,4095.0,0.0,418.0,1608.0,590.0,1164.0,0.0,0.0,0.0,0.0,NaN,GBR,1,0
6,0,1,1,1,19,19,2,0,1,2,NaN,963.0,1657.0,4095.0,4095.0,671.0,684.0,600.0,798.0,830.0,648.0,0.0,0.0,4095.0,0.0,4095.0,0.0,418.0,1608.0,590.0,1164.0,0.0,0.0,0.0,0.0,NaN,SWE,0,0
7,0,1,1,1,20,27,2,0,1,4,NaN,963.0,1657.0,4095.0,4095.0,671.0,684.0,600.0,798.0,830.0,648.0,0.0,0.0,4095.0,0.0,4095.0,0.0,418.0,1608.0,590.0,1164.0,671.0,850.0,0.0,0.0,NaN,GBR,1,0
8,0,1,1,1,21,19,1,2,0,1,NaN,963.0,1657.0,4095.0,4095.0,671.0,684.0,600.0,798.0,830.0,648.0,866.0,1628.0,4095.0,0.0,4095.0,0.0,418.0,1608.0,590.0,1164.0,671.0,850.0,0.0,0.0,NaN,SWE,0,0
9,0,1,1,1,22,27,1,0,0,0,NaN,963.0,1657.0,4095.0,4095.0,671.0,684.0,600.0,798.0,830.0,648.0,866.0,1628.0,4095.0,0.0,4095.0,0.0,418.0,1608.0,590.0,1164.0,671.0,850.0,1031.0,496.0,NaN,GBR,1,0


### 점수차 기반 파워플레이 전략 분석
-엔드 시작 시 점수차 계산

In [264]:
# 0) 정렬(안전)
ends_full = ends_full.sort_values(
    ["CompetitionID","SessionID","GameID","EndID","TeamID"]
).reset_index(drop=True)

# 1) 팀별 누적 점수(엔드 종료 시점 기준) 계산
ends_full["CumScore"] = (
    ends_full
    .groupby(["CompetitionID","SessionID","GameID","TeamID"])["Result"]
    .cumsum()
)

# 2) 같은 (경기, 엔드)에서 '상대' 팀의 이번 엔드 득점/누적을 조인으로 붙이기
opp = (
    ends_full[["CompetitionID","SessionID","GameID","EndID","TeamID","Result","CumScore"]]
    .rename(columns={
        "TeamID":   "TeamID_opp",
        "Result":   "OppResult",
        "CumScore": "OppCumScore_exact"
    })
)

ends_join = ends_full.merge(
    opp,
    on=["CompetitionID","SessionID","GameID","EndID"],
    how="left"
)

# 3) 자기 자신은 제거하여 '진짜 상대'만 남기기
ends_join = ends_join[ends_join["TeamID"] != ends_join["TeamID_opp"]].copy()

# 4) 엔드 '시작' 시 누적 점수 (현재 누적에서 이번 엔드 득점을 빼면 시작 시점)
ends_join["CumScoreStart"]      = ends_join["CumScore"]             - ends_join["Result"]
ends_join["OppCumScoreStart"]   = ends_join["OppCumScore_exact"]    - ends_join["OppResult"]

# 5) 엔드 시작 시 점수차 & 엔드 스윙
ends_join["ScoreDiffStart"] = ends_join["CumScoreStart"] - ends_join["OppCumScoreStart"]
# 엔드 스윙은 이번 엔드의 순득점: 우리 득점 - 상대 득점
ends_join["EndSwing"]       = ends_join["Result"] - ends_join["OppResult"]

# 6) (검증) 종료후 점수차 = 시작점수차 + 엔드스윙
# ends_join["ScoreDiff_check"] = ends_join["ScoreDiffStart"] + ends_join["EndSwing"]

# 7) 파워플레이를 실제로 '사용한 팀'만 남기고, (경기,엔드)당 1행으로 축약
pp_end_rows = (
    ends_join[ends_join["PowerPlay"].isin([1, 2])]
      .sort_values(["CompetitionID","SessionID","GameID","EndID","TeamID"])
      .groupby(["CompetitionID","SessionID","GameID","EndID"], as_index=False)
      .first()
)

In [265]:
pp_end_rows.head()

,CompetitionID,SessionID,GameID,EndID,TeamID,Result,PowerPlay,GroupID,Sheet,NOC1,NOC2,ResultStr1,ResultStr2,LSFE,Winner,TeamID1,TeamID2,NOC,Name,CumScore,TeamID_opp,OppResult,OppCumScore_exact,CumScoreStart,OppCumScoreStart,ScoreDiffStart,EndSwing
0,0,1,1,6,27,1,1.0,0,A,SWE,GBR,5,9,0,0,19,27,GBR,Great Britain,5,19,0,5,4,5,-1,1
1,0,1,1,7,19,0,1.0,0,A,SWE,GBR,5,9,0,0,19,27,SWE,Sweden,5,27,3,8,5,5,0,-3
2,0,1,2,6,46,3,2.0,0,B,AUS,USA,5,6,1,0,46,20,AUS,Australia,5,20,0,3,2,3,-1,3
3,0,1,2,7,20,2,2.0,0,B,AUS,USA,5,6,1,0,46,20,USA,United States of America,5,46,0,5,3,5,-2,2
4,0,1,3,6,17,2,1.0,0,C,NOR,CZE,6,7,0,0,17,22,NOR,Norway,5,22,0,3,3,3,0,2


- 샷 품질(Points) 피처

In [266]:
pp_shots = stones_full[stones_full["PowerPlay"].isin([1,2])].copy()

shot_quality = (
    pp_shots.groupby(["CompetitionID","SessionID","GameID","EndID","TeamID"])
    .agg(
        AvgPoints=("Points", "mean"),
        MissRate=("Points", lambda x: (x==0).mean()),
        Aggressiveness=("Task",  lambda x: x.isin([6,7,9,10]).mean()) # Take-out/Hit&Roll/Double/Promotion
    )
    .reset_index()
)

In [269]:
shot_quality.head(16)

,CompetitionID,SessionID,GameID,EndID,TeamID,AvgPoints,MissRate,Aggressiveness
0,0,1,1,6,27,3.0,0.0,0.2
1,0,1,1,7,19,1.2,0.6,0.6
2,0,1,2,6,46,3.2,0.0,0.0
3,0,1,2,7,20,3.0,0.0,0.6
4,0,1,3,6,17,3.8,0.0,0.4
5,0,1,3,7,22,3.4,0.0,0.4
6,0,1,4,6,18,2.2,0.4,0.4
7,0,1,4,7,43,3.2,0.0,0.2
8,0,2,1,6,46,3.4,0.0,0.0
9,0,2,1,7,43,2.8,0.2,0.2


- 공간 패턴(Stone_1) 피처

  
  PowerPlay는 초기 배치가 전략의 핵심이라 Stone_1로 측정.
(행이 샷별이므로, 동일 엔드·팀에서 한 번만 집계되도록 먼저 대표행을 잡는다.)

In [272]:
# 엔드·팀 대표행(최초 샷행) 기준으로 Stone_1 좌표 취득
pp_pos = (
    pp_shots.sort_values(["CompetitionID","SessionID","GameID","EndID","TeamID","ShotID"])
            .groupby(["CompetitionID","SessionID","GameID","EndID","TeamID"], as_index=False)
            .first()[["CompetitionID","SessionID","GameID","EndID","TeamID","stone_1_x","stone_1_y"]]
)

# 버튼(750,800)과의 거리
pp_pos["DistToButton"] = np.sqrt( (pp_pos["stone_1_x"]-750)**2 + (pp_pos["stone_1_y"]-800)**2 )

In [274]:
pp_pos.head(10)

,CompetitionID,SessionID,GameID,EndID,TeamID,stone_1_x,stone_1_y,DistToButton
0,0,1,1,6,27,1089.0,1916.0,1166.352005
1,0,1,1,7,19,1235.0,603.0,523.482569
2,0,1,2,6,46,350.0,850.0,403.112887
3,0,1,2,7,20,577.0,1693.0,909.603210
4,0,1,3,6,17,1150.0,850.0,403.112887
5,0,1,3,7,22,1089.0,1916.0,1166.352005
6,0,1,4,6,18,1089.0,1916.0,1166.352005
7,0,1,4,7,43,350.0,850.0,403.112887
8,0,2,1,6,46,1193.0,723.0,449.642080
9,0,2,1,7,43,1089.0,1916.0,1166.352005


한 엔드 단위로 합치기 (상황 + 샷품질 + 공간)

In [275]:
pp_end_features = (
    pp_ends[[
        "CompetitionID","SessionID","GameID","EndID","TeamID","NOC",
        "EndSwing","ScoreDiffStart","ScoreDiff"  # 효과/상황
    ]]
    .merge(shot_quality, on=["CompetitionID","SessionID","GameID","EndID","TeamID"], how="left")
    .merge(pp_pos,      on=["CompetitionID","SessionID","GameID","EndID","TeamID"], how="left")
)

# 안전 필터: 값 없는 행 제거
pp_end_features = pp_end_features.dropna(subset=["EndSwing","AvgPoints","MissRate","Aggressiveness","DistToButton"])

In [277]:
pp_end_features.head()

,CompetitionID,SessionID,GameID,EndID,TeamID,NOC,EndSwing,ScoreDiffStart,ScoreDiff,AvgPoints,MissRate,Aggressiveness,stone_1_x,stone_1_y,DistToButton
0,0,1,1,6,27,GBR,1.0,-1.0,0.0,3.0,0.0,0.2,1089.0,1916.0,1166.352005
1,0,1,1,7,19,SWE,-3.0,0.0,-3.0,1.2,0.6,0.6,1235.0,603.0,523.482569
2,0,1,2,6,46,AUS,1.0,-1.0,0.0,3.2,0.0,0.0,350.0,850.0,403.112887
3,0,1,2,7,20,USA,2.0,-2.0,0.0,3.0,0.0,0.6,577.0,1693.0,909.603210
4,0,1,3,6,17,NOR,2.0,0.0,2.0,3.8,0.0,0.4,1150.0,850.0,403.112887


5) 나라×엔드 수준으로 요약

   엔드별 빈도가 너무 적으면 과적합/노이즈라서 최소 표본수 임계치 두는 걸 추천(예: 3경기 이상).

In [278]:
pp_by_noc_end = (
    pp_end_features.groupby(["NOC","EndID"])
    .agg(
        n=("GameID","nunique"),
        AvgEndSwing=("EndSwing","mean"),          # PP 사용 엔드의 평균 순득점(효과)
        AvgPoints=("AvgPoints","mean"),           # 샷 품질
        MissRate=("MissRate","mean"),             # 실패율 (낮을수록 좋음)
        Aggressiveness=("Aggressiveness","mean"), # 공격성
        MeanDist=("DistToButton","mean"),         # 버튼과 평균 거리 (낮을수록 중앙 집중)
        DistVar=("DistToButton","var")            # 거리 분산 (낮을수록 밀집)
    )
    .reset_index()
)

# 표본 부족 제거(필요 시 조정)
min_games = 3
pp_by_noc_end = pp_by_noc_end[pp_by_noc_end["n"] >= min_games].copy()

6) 통합 ‘효과’ 점수 만들기 (표준화 + 가중합)

   척도 다르므로 표준화(z-score) 후 방향 맞춰 가중합.

In [279]:
features = ["AvgEndSwing","AvgPoints","MissRate","Aggressiveness","MeanDist","DistVar"]
Z = pp_by_noc_end[features].copy()

# 방향 통일: '작을수록 좋은' 항목은 부호 뒤집기
# - MissRate, MeanDist, DistVar 는 낮을수록 좋음 → 음수화
Z["MissRate"]  = -Z["MissRate"]
Z["MeanDist"]  = -Z["MeanDist"]
Z["DistVar"]   = -Z["DistVar"]

# 표준화
scaler = StandardScaler()
Z_scaled = pd.DataFrame(scaler.fit_transform(Z), columns=features, index=pp_by_noc_end.index)

# 가중치 설정(원하면 조정 가능)
w = {
    "AvgEndSwing": 0.35,  # 실제 점수 변화에 가장 큰 가중
    "AvgPoints":   0.25,  # 샷 품질
    "MissRate":    0.15,  # 실패 억제
    "Aggressiveness":0.10,# 팀 성향 반영
    "MeanDist":    0.10,  # 중앙 집중
    "DistVar":     0.05   # 공간 밀집
}

pp_by_noc_end["PP_Effectiveness"] = sum(Z_scaled[col]*w[col] for col in features)

7) “나라별 최적 엔드” 도출

In [280]:
best_per_noc = (
    pp_by_noc_end.loc[pp_by_noc_end.groupby("NOC")["PP_Effectiveness"].idxmax(),
                      ["NOC","EndID","PP_Effectiveness","n",
                       "AvgEndSwing","AvgPoints","MissRate","Aggressiveness","MeanDist","DistVar"]]
    .sort_values(["PP_Effectiveness"], ascending=False)
    .reset_index(drop=True)
)

# 보기 좋게 반올림
num_cols = ["PP_Effectiveness","AvgEndSwing","AvgPoints","MissRate","Aggressiveness","MeanDist","DistVar"]
best_per_noc[num_cols] = best_per_noc[num_cols].round(3)

best_per_noc.head(20)

,NOC,EndID,PP_Effectiveness,n,AvgEndSwing,AvgPoints,MissRate,Aggressiveness,MeanDist,DistVar
0,USA,8,1.354,3,7.750,3.000,0.100,0.600,669.315,116153.414
1,ITA,8,1.022,4,8.750,3.000,0.100,0.200,804.328,186305.237
2,AUS,8,1.010,3,7.500,2.900,0.100,0.450,784.732,194177.984
3,KOR,8,1.007,5,10.429,2.829,0.200,0.371,948.196,138810.498
4,CAN,6,0.933,5,2.400,3.600,0.000,0.280,630.020,115740.025
5,SCO,6,0.761,5,0.889,3.511,0.022,0.511,707.362,159436.815
6,CZE,7,0.532,3,0.714,3.400,0.029,0.486,839.250,166438.272
7,JPN,7,0.448,5,1.727,3.164,0.036,0.345,617.038,124719.742
8,NOR,7,0.441,5,0.583,3.400,0.050,0.317,532.666,90460.591
9,SWE,6,0.273,5,0.812,3.475,0.062,0.400,967.610,1121639.440


In [283]:
# === Power Play 효과 시각화(Heatmap / Bar / Scatter) ===
# 필요 DF:
#   1) pp_by_noc_end : (NOC, EndID)별 요약
#        ["NOC","EndID","n","PP_Effectiveness","AvgEndSwing","AvgPoints","MissRate","Aggressiveness","MeanDist","DistVar"]
#   2) best_per_noc  : 나라별 최적 엔드
#        ["NOC","EndID","PP_Effectiveness","n","AvgEndSwing","AvgPoints","MissRate","Aggressiveness","MeanDist","DistVar"]

missing = [name for name in ["pp_by_noc_end","best_per_noc"] if name not in globals()]
if missing:
    raise RuntimeError(f"세션에 {missing} DataFrame이 없습니다. 먼저 계산 후 실행하세요.")

# 정리
df_all  = pp_by_noc_end.copy().sort_values(["NOC","EndID"]).reset_index(drop=True)
df_best = best_per_noc.copy().sort_values(["PP_Effectiveness"], ascending=False).reset_index(drop=True)

os.makedirs("figs", exist_ok=True)

# 1) Heatmap: NOC × EndID 의 PP_Effectiveness
pivot = df_all.pivot_table(index="NOC", columns="EndID", values="PP_Effectiveness", aggfunc="mean")
fig, ax = plt.subplots(figsize=(max(8, pivot.shape[1]*0.6), max(6, pivot.shape[0]*0.45)))
im = ax.imshow(pivot.values, aspect="auto")
ax.set_yticks(np.arange(pivot.shape[0])); ax.set_yticklabels(pivot.index.tolist())
ax.set_xticks(np.arange(pivot.shape[1])); ax.set_xticklabels(pivot.columns.tolist())
ax.set_xlabel("EndID"); ax.set_ylabel("NOC")
ax.set_title("Power Play Effectiveness (by NOC × End)")
# 값 주석(셀 수가 너무 많지 않을 때만)
if pivot.size <= 300:
    for i in range(pivot.shape[0]):
        for j in range(pivot.shape[1]):
            val = pivot.values[i, j]
            if not (isinstance(val, float) and (np.isnan(val) or np.isinf(val))):
                ax.text(j, i, f"{val:.2f}", ha="center", va="center", fontsize=8)
fig.tight_layout()
fig.savefig("figs/pp_effect_heatmap.png", dpi=150)
plt.close(fig)

# 2) Bar: 나라별 최적 엔드 효과 비교
fig, ax = plt.subplots(figsize=(10, max(6, len(df_best)*0.35)))
y = np.arange(len(df_best))
ax.barh(y, df_best["PP_Effectiveness"].values)
ax.set_yticks(y)
ax.set_yticklabels([f"{row.NOC} (End {int(row.EndID)})" for row in df_best.itertuples()])
ax.set_xlabel("PP_Effectiveness (standardized composite)")
ax.set_title("Best Power Play Timing per Country")
for i, v in enumerate(df_best["PP_Effectiveness"].values):
    ax.text(v, i, f" {v:.2f}", va="center", fontsize=8)
fig.tight_layout()
fig.savefig("figs/pp_best_end_bar.png", dpi=150)
plt.close(fig)

# 3) Scatter: 엔드 스윙 vs 샷 품질 (버블=표본수), 라벨=NOC-End
fig, ax = plt.subplots(figsize=(9, 7))
n_min, n_max = df_all["n"].min(), df_all["n"].max()
sizes = np.full(len(df_all), 120.0) if n_min==n_max else 80 + 320*(df_all["n"]-n_min)/(n_max-n_min)
ax.scatter(df_all["AvgEndSwing"].values, df_all["AvgPoints"].values, s=sizes, alpha=0.7)
ax.set_xlabel("AvgEndSwing (점수차 변화, +가 유리)")
ax.set_ylabel("AvgPoints (샷 품질)")
ax.set_title("Effect vs Quality per (NOC × End) — bubble size = n")
# 혼잡 줄이기: PP_Effectiveness 상위 일부만 라벨
try:
    top_idx = df_all["PP_Effectiveness"].nlargest(min(30, len(df_all))).index
    for idx in top_idx:
        r = df_all.loc[idx]
        ax.annotate(f"{r.NOC}-E{int(r.EndID)}", (r["AvgEndSwing"], r["AvgPoints"]),
                    xytext=(3,3), textcoords="offset points", fontsize=8)
except Exception:
    pass
fig.tight_layout()
fig.savefig("figs/pp_effect_scatter.png", dpi=150)
plt.close(fig)

print("✅ 생성된 파일")
print(" - figs/pp_effect_heatmap.png")
print(" - figs/pp_best_end_bar.png")
print(" - figs/pp_effect_scatter.png")


✅ 생성된 파일
 - figs/pp_effect_heatmap.png
 - figs/pp_best_end_bar.png
 - figs/pp_effect_scatter.png
